<a href="https://colab.research.google.com/github/TBLinhDan/RAG_on_PDF_Q-A_using_LLM/blob/feature%2Fmodule1-week4-RAG_on_PDF_Q%26A_using_LLM/RAG_on_PDF_Q%26A_using_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Sử dụng thư viện LangChain xây dựng một chương trình RAG với các bước xử lý cơ bản.**  
Thư viện LangChain là một thư viện được thiết kế chuyên biệt cho việc xây dựng
các ứng dụng về LLMs.

In [ ]:
# 1. Cài đặt các gói thư viện cần thiết:

!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

**2. Xây dựng Vector Database:**  
Với dữ liệu nguồn là một file pdf, thực hiện đưa các nội dung trong file này vào cơ sở dữ liệu, với các bước:  
***file.pdf --> text Splitter --> Vectorrization --> Vector database***

In [ ]:
# Import các thư viện cần thiết:

import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [ ]:
# sử dụng class PyPDFLoader để đọc file pdf đầu vào:
Loader = PyPDFLoader
FILE_PATH = "./YOLOv10_Tutorials.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

In [ ]:
# Khởi tạo bộ tách văn bản (text splitter), tách file.pdf ra thành các đoạn văn bản nhỏ
# mỗi đoạn văn bản nhỏ được coi như là một tài liệu trong cơ sở dữ liệu.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

print ("Number of sub-documents:", len(docs)) # số lượng tài liệu
print (docs[0])		# nội dung tài liệu đầu tiên


Number of sub-documents: 33
page_content='AI VIET NAM – AI COURSE 2024\nTutorial: Phát hiện đối tượng trong ảnh với\nYOLOv10\nDinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và\nQuang-Vinh Dinh\nNgày 20 tháng 6 năm 2024\nI. Giới thiệu\nObject Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh vực\nComputer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong\nmột tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm\ngiải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look\nOnce) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực\nthi mà loại mô hình này mang lại.\nHình 1: Logo của mô hình YOLO. Ảnh: link.\nThời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)\nđã đề xuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object\nDetection [10]. Với những cải tiến mới, mô hình đã đạt đượ

**Number of sub-documents:** 33  
page_content='AI VIET NAM – AI COURSE 2024\nTutorial:  
Phát hiện đối tượng trong ảnh với\nYOLOv10\nDinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và\nQuang-Vinh Dinh\nNgày 20 tháng 6 năm 2024\nI.  
Giới thiệu\nObject Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh vực\nComputer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong\nmột tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm\ngiải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look\nOnce) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực\nthi mà loại mô hình này mang lại.\n  
Hình 1: Logo của mô hình YOLO. Ảnh: link.\n  
Thời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)\nđã đề xuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object\nDetection [10]. Với những cải tiến mới, mô hình đã đạt được hiệu suất vượt trội hơn so với các  
' metadata={'source': './YOLOv10_Tutorials.pdf', 'page': 0}


In [ ]:
# Khởi tạo instance vectorization, chuyển đổi các văn bản thành các vector, giúp dễ dàng và chính xác trong thục hiện truy vấn.

embedding = HuggingFaceEmbeddings() # chuyển đổi văn bản thành vector

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#  Khởi tạo vector database từ text_splitter các đoạn văn ản nhỏ đã được Vectorrization qua embedding

vector_db = Chroma.from_documents(documents=docs, embedding=embedding)
retriever = vector_db.as_retriever()


In [ ]:
# Thử thực hiện truy vấn với một đoạn văn bản bất kì.
# (Kết quả sẽ trả về cho ta danh sách các tài liệu có liên quan đến câu hỏi đầu vào)

result = retriever.invoke("What is YOLO?")
print("Number of relevant documents:", len(result))


Number of relevant documents: 4


**3. Khởi tạo mô hình ngôn ngữ lớn, sử dụng mô hình Vicuna.**  
Vicuna là mô hình LLM nguồn mở, có hiệu suất rất ổn, và phản hồi tốt với tiếng Việt

In [ ]:
# Khai báo một số cài đặt cần thiết cho mô hình:

nf4_config = BitsAndBytesConfig(
	    load_in_4bit=True,
	    bnb_4bit_quant_type="nf4",
	    bnb_4bit_use_double_quant=True,
	    bnb_4bit_compute_dtype=torch.bfloat16
	    )


In [ ]:
# Khởi tạo mô hình và tokenizer:

MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
	      MODEL_NAME,
	      quantization_config=nf4_config,
	      low_cpu_mem_usage = True
	      )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
# Tích hợp tokenizer và model thành một pipeline để tiện sử dụng:

model_pipeline = pipeline(
	      "text-generation",
	      model=model,
	      tokenizer=tokenizer,
	      max_new_tokens=512,
	      pad_token_id=tokenizer.eos_token_id,
	      device_map="auto"
	      )

llm = HuggingFacePipeline(pipeline = model_pipeline,)

**4. Kết hợp vector database, retriever và mô hình Vicuna để hoàn thành chương trình RAG, có khả năng hỏi đáp các nội dung trong một file pdf.**

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
	      {"context": retriever | format_docs, 	"question": RunnablePassthrough
	      ()}
	      | prompt
	      | llm
	      | StrOutputParser()
)

USER_QUESTION = "YOLOv10 là gì?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer:')[1].strip()
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


YOLOv10 là một phiên bản của YOLO (You Only Look Once) - một hệ thống dự đoán hình ảnh được huấn luyện sẵn trên bộ dữ liệu COCO. Phiên bản này được tạo bằng cách tải về trọng số (file.pt) từ GitHub và khởi tạo mô hình bằng cách sử dụng thư viện ultralytics.


**Question:** YOLOv10 là gì?

**Answer:**  
YOLOv10 là một phiên bản của YOLO (You Only Look Once) - một hệ thống dự đoán hình ảnh được huấn luyện sẵn trên bộ dữ liệu COCO. Phiên bản này được tạo bằng cách tải về trọng số (file.pt) từ GitHub và khởi tạo mô hình bằng cách sử dụng thư viện ultralytics.


In [ ]:
USER_QUESTION = "So sánh các phiên bản YOLO?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer:')[1].strip()
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


YOLOv9 có mức độ tối ưu hơn về mặt tham số mô hình và độ trễ trong inference so với các phiên bản trước, nhưng vẫn giữ được độ chính xác ngang hoặc hơn so với các phiên bản trước. YOLOv9 sử dụng PGI và GELAN để cải thiện độ chính xác và hiệu suất của mô hình. YOLOv9 có tốc độ nhanh hơn so với YOLOv8 nhờ các cải tiến trong kiến trúc mạng. Tuy nhiên, YOLOv9 vẫn yêu cầu nhiều tài nguyên tính toán và gặp khó khăn trong việc phát hiện các object rất nhỏ.


**Question:** So sánh các phiên bản YOLO?

**Answer:**  
YOLOv9 có mức độ tối ưu hơn về mặt tham số mô hình và độ trễ trong inference so với các phiên bản trước, nhưng vẫn giữ được độ chính xác ngang hoặc hơn so với các phiên bản trước. YOLOv9 sử dụng PGI và GELAN để cải thiện độ chính xác và hiệu suất của mô hình. YOLOv9 có tốc độ nhanh hơn so với YOLOv8 nhờ các cải tiến trong kiến trúc mạng. Tuy nhiên, YOLOv9 vẫn yêu cầu nhiều tài nguyên tính toán và gặp khó khăn trong việc phát hiện các object rất nhỏ.


In [ ]:
USER_QUESTION = "Object Detection là gì ?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer:')[1].strip()
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Object Detection là một bài toán trong Computer Vision, nó được xác định bằng việc tự động xác định vị trí và nhận diện tên (class) của các vật thể trong một bức ảnh. Có ba hướng tiếp cận cho bài toán này: One-stage Object Detection, Two-stage Object Detection và End-to-end Object Detection. Các phiên bản YOLO là một trong những mô hình one-stage Object Detection được sử dụng rộng rãi.


**Question:** Object Detection ?

**Answer:**  
Object Detection is a computer vision problem that aims to build a program that can automatically detect and recognize the position and class of objects in an image. The input is a picture, and the output is a bounding box of the detected objects. There are three approaches in object detection: one-stage, two-stage, and end-to-end. YOLOv1 and YOLOv2 are examples of one-stage object detection.

**Question:** Object Detection là gì ?

**Answer:**  
Object Detection là một bài toán trong Computer Vision, nó được xác định bằng việc tự động xác định vị trí và nhận diện tên (class) của các vật thể trong một bức ảnh. Có ba hướng tiếp cận cho bài toán này: One-stage Object Detection, Two-stage Object Detection và End-to-end Object Detection. Các phiên bản YOLO là một trong những mô hình one-stage Object Detection được sử dụng rộng rãi.
